In [ ]:
**Translate English Audio to Hindi (After 6 PM IST) AND Translate English Words to Hindi with Vowel Restrictions (Between 9-10 PM IST)**

In [ ]:
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer
from datetime import datetime
import pytz
import speech_recognition as sr  


model_name_hi = 'Helsinki-NLP/opus-mt-en-hi'

try:
    tokenizer_hi = MarianTokenizer.from_pretrained(model_name_hi)
    model_hi = MarianMTModel.from_pretrained(model_name_hi)
except Exception as e:
    print(f"Error loading models: {e}")


def get_current_ist_time():
    tz_ist = pytz.timezone('Asia/Kolkata')
    return datetime.now(tz_ist).time()


def translate_audio_to_hindi(audio):
    current_time = get_current_ist_time()

    
    if current_time.hour >= 18:
        
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio) as source:
            audio_data = recognizer.record(source)
            try:
                transcript = recognizer.recognize_google(audio_data)
                inputs_hi = tokenizer_hi(transcript, return_tensors="pt", padding=True)
                translated_tokens_hi = model_hi.generate(**inputs_hi)
                hindi_translation = tokenizer_hi.decode(translated_tokens_hi[0], skip_special_tokens=True)
                return f"Translated to Hindi: {hindi_translation}"
            except sr.UnknownValueError:
                return "Sorry, I could not understand the audio."
            except sr.RequestError as e:
                return f"Error with Speech Recognition: {e}"
    else:
        return "Please try after 6 PM IST."


def translate_word_to_hindi(word):
    current_time = get_current_ist_time()

    vowels = 'AEIOUaeiou'
    if word[0] in vowels:
        # Check if current time is between 9-10 PM IST
        if current_time.hour == 21:
            # Allow translation during the permitted time
            inputs_hi = tokenizer_hi(word, return_tensors="pt", padding=True)
            translated_tokens_hi = model_hi.generate(**inputs_hi)
            hindi_translation = tokenizer_hi.decode(translated_tokens_hi[0], skip_special_tokens=True)
            return f"Translated to Hindi: {hindi_translation}"
        else:
            return f"The word '{word}' starts with a vowel. Translation is not allowed except between 9-10 PM IST."
    else:
        # Translate normally if the word starts with a consonant
        inputs_hi = tokenizer_hi(word, return_tensors="pt", padding=True)
        translated_tokens_hi = model_hi.generate(**inputs_hi)
        hindi_translation = tokenizer_hi.decode(translated_tokens_hi[0], skip_special_tokens=True)
        return f"Translated to Hindi: {hindi_translation}"


def gradio_interface_audio(audio):
    return translate_audio_to_hindi(audio)


def gradio_interface_word(word):
    return translate_word_to_hindi(word)


interface_audio = gr.Interface(
    fn=gradio_interface_audio,
    inputs=gr.Audio(type="filepath", label="Upload or Record English Audio"),  # Updated Audio component
    outputs="text",
    title="English Audio to Hindi Translation (After 6 PM IST)",
    description="Upload or record English audio and get its translation to Hindi if it's after 6 PM IST."
)

interface_word = gr.Interface(
    fn=gradio_interface_word,
    inputs=gr.Textbox(lines=1, placeholder="Enter an English word"),
    outputs="text",
    title="English to Hindi Translation with Vowel Restrictions",
    description="Translate English words to Hindi, but words starting with vowels are restricted except between 9-10 PM IST."
)


interface_audio.launch()
interface_word.launch()
